# AIMM post processing package
To install the package, 

`git clone git@github.com:zhul9311/aimm-post-processing.git`

In the package folder, run

 `python setup.py develop`

# Import modules and load data from TILED

In [ ]:
# from os import remove
# from numpy import identity
import sys
import tiled
from tiled.client import from_uri
from aimm_post_processing.operations import (
    Pull,
    Identity,
    RemoveBackground, 
    StandardizeGrid,
    StandardizeIntensity
)
from aimm_post_processing.pipeline import Pipeline

# %reload_ext autoreload
# %autoreload 1
# %aimport aimm_post_processing.pipeline
# %aimport aimm_post_processing.operations
# %aimport isstools.xasproject.xasproject

In [100]:
client = from_uri("https://aimm.lbl.gov/api")
view = client['dataset']['newville']['uid']["tFrGpD3A8BX"]

In [ ]:
# This data is SrCO3 K-edge on Sr
view.metadata

# Operation Chain (Pipeline)

In [111]:
pull = Pull()
identity = Identity()
standardizegrid = StandardizeGrid(
    x0=15700, 
    xf=17000, 
    nx=1301,
    x_column="energy",
    y_columns=["mutrans", "mufluor", "murefer"]
)
removebackground = RemoveBackground(
    x0=15700, 
    xf=15800, 
    x_column="energy",
    y_columns=["mutrans", "mufluor", "murefer"],
    victoreen_order=0
)
standardizeintensity = StandardizeIntensity(
    x0=15700,
    xf=15800,
    x_column="energy",
    y_columns=["mutrans", "mufluor", "murefer"]
)

In [112]:
pipe0 = Pipeline(
    [
        pull,
        standardizegrid,
        removebackground
    ]
)

In [113]:
d = pipe0.run(view)

In [ ]:
d["data"].plot(x="energy", y=["mutrans", "mufluor", "murefer"])

In [ ]:
d["metadata"]["post_processing"]

# Another example of Pipeline

In [118]:
pipe1 = Pipeline(
    [
        identity,
        standardizegrid,
        standardizeintensity
    ]
)

In [119]:
d = pipe1.run(d)

In [ ]:
d["data"].plot(x="energy", y=["mutrans", "mufluor", "murefer"])

In [ ]:
d["metadata"]["post_processing"]

# Pipeline concatenation

In [122]:
pipe2 = Pipeline(
    [
        pull,
        standardizegrid,
    ]
)
pipe3 = Pipeline(
    [
        removebackground,
        standardizeintensity
    ]
)
pipe4 = pipe2 + pipe3

In [123]:
d = pipe4.run(view)

In [ ]:
d["data"].plot(x="energy", y=["mutrans", "mufluor", "murefer"])

In [ ]:
d["metadata"]["post_processing"]

# User-defined operator

In [126]:
# All operators are defined based on Operator class
from aimm_post_processing.operations import Operator


### The template for creating user-defined operator ### 
class MyOperator(Operator):
    # Replace `MyOperaor` with your favorate name.
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs) 
        # *args and **kwargs are user defined parameters needed for the operator.
        # all extra args and kwargs must also defined as local attributes if you want
        # to use them in `_process_data` function.
        # E.g. If you have __init__(self, x, y=5, x_column="energy", y_columns=["mu"])
        #      in definition, you must also have self.x = x and self.y = y
        #      "x_column" and "y_columns" ar already implemented automatically, so you
        #.     don't need to worry about them.

    def _process_data(self, df):
        """
        Parameters
        ----------
        df : pandas.DataFrame
            The dataframe that has columns such as "energy", "mu", etc.

        Returns
        -------
        df : pandas.DataFrame
            Processed dataframe
        """
        ### Write your code here ###
        
        new_df = df # this line is to be replaced
        
        return new_df


### An example implementation ###

class Shift(Operator):
    """This example shifts the energy grid by x
    """
    def __init__(self, x, x_column="energy", y_columns=["mu"]):
        super().__init__(x_column, y_columns)
        self.x = x # must have this line.
        
    def _process_data(self, df):
        shift = self.x
        energy_grid = df[self.x_column]
        df[self.x_column] = energy_grid + shift
        return df

In [128]:
client = from_uri("https://aimm.lbl.gov/api")
view = client['dataset']['newville']['uid']["tFrGpD3A8BX"]

In [ ]:
df = view.read()
df.plot(x="energy", y=["mutrans", "mufluor", "murefer"])

In [130]:
pull = Pull()
shift_operator = Shift(1000)

In [131]:
data_dict = pull(view)
new_dict = shift_operator(data_dict)

In [ ]:
new_dict["data"].plot(x="energy", y=["mutrans", "mufluor", "murefer"])

In [ ]:
new_dict["metadata"]["post_processing"]